classifiers: nn.crossentropyloss = -log-likelihood --- use for logp(y) and -logq(y/x) for support <br>
kl-div: <br>
reconstr-loss: set reduction to none and then take mean of losses per image in the total batch. This gives reconstr-loss per image for further computation<br> 

In [1]:
import sys
import random
import numpy as np
import torch
import learn2learn as l2l
import pandas as pd

from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from PIL.Image import LANCZOS
import wandb

In [2]:
torch.cuda.is_available()

True

In [3]:
sys.path.append('/home/anuj/Desktop/Work/TU_Delft/research/implement/learning_to_meta-learn')
sys.path

['/home/anuj/Desktop/Work/TU_Delft/research/implement/learning_to_meta-learn/src',
 '/home/anuj/.vscode/extensions/ms-toolsai.jupyter-2021.9.1001312534/pythonFiles',
 '/home/anuj/.vscode/extensions/ms-toolsai.jupyter-2021.9.1001312534/pythonFiles/lib/python',
 '/home/anuj/anaconda3/envs/torch/lib/python38.zip',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8/lib-dynload',
 '',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages/datasets-1.2.1-py3.8.egg',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages/locket-0.2.1-py3.8.egg',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages/IPython/extensions',
 '/home/anuj/.ipython',
 '/home/anuj/Desktop/Work/TU_Delft/research/implement/learning_to_meta-learn']

In [4]:
#from data.loaders import Omniglotmix, MiniImageNet
from data.taskers import gen_tasks
from src.zoo.archs import CCVAE, CEncoder, Classifier_VAE
from src.zoo.delpo_utils import setup, inner_adapt_delpo, loss, accuracy

#from src.utils import Profiler

In [5]:
train_tasks = gen_tasks(dataname='miniimagenet', root='../../dataset/mini_imagenet', mode='train', n_ways=5, k_shots=1, q_shots=1, image_transforms=None)

KeyboardInterrupt: 

In [21]:
classes = list(range(1623))
random.shuffle(classes)
image_transforms = transforms.Compose([transforms.Resize(28, interpolation=LANCZOS),
                                                    transforms.ToTensor(),
                                                    lambda x: 1.0 - x,
                                                ])
train_tasks = gen_tasks('omniglot', '/home/anuj/Desktop/Work/TU_Delft/research/implement/omniglot', image_transforms=image_transforms, n_ways=5, k_shots=5, q_shots=5, classes=classes[:1100])

/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages/torchvision/transforms/transforms.py:280: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


In [6]:
task = train_tasks.sample()

In [5]:
device = 'cpu'
n_ways = 5
k_shots = 5
q_shots = 15
dataset = 'miniimagenet'
root = '../../dataset/mini_imagenet'
order = False
inner_lr = 0.0001
meta_lr = 0.0001
reconst_loss = nn.MSELoss(reduction='none')
inner_adapt_steps_train = 5

In [6]:
train_tasks, valid_tasks, test_tasks, learner = setup(
    dataset, root, n_ways, k_shots, q_shots, order, inner_lr, device, download='False')
opt = optim.Adam(learner.parameters(), meta_lr)

In [7]:
opt.zero_grad()
meta_train_loss = []
meta_valid_loss = []
meta_train_acc = []
meta_valid_acc = []


In [8]:
task = train_tasks.sample()
model = learner.clone()
data, labels = task
data, labels = data.to(device) / 256, labels.to(device)
total = n_ways * (k_shots + q_shots)
queries_index = np.zeros(total)

# Extracting the evaluation datums from the entire task set, for the meta gradient calculation
for offset in range(n_ways):
    queries_index[np.random.choice(
        k_shots+q_shots, q_shots, replace=False) + ((k_shots + q_shots)*offset)] = True
support = data[np.where(queries_index == 0)]
support_labels = labels[np.where(queries_index == 0)]
queries = data[np.where(queries_index == 1)]
queries_labels = labels[np.where(queries_index == 1)]


In [9]:
model = learner.clone()

In [22]:
reconst_image, logits, mu_l, log_var_l, mu_s, log_var_s = model(support)
adapt_loss = loss(reconst_loss, reconst_image, support, logits, support_labels, mu_s, log_var_s, mu_l, log_var_l)
adapt_loss

(tensor(1137.4082, grad_fn=<AddBackward0>),
 tensor(56.8141, grad_fn=<MeanBackward0>),
 tensor(25.3008, grad_fn=<MeanBackward0>),
 tensor(2422.6084, grad_fn=<MeanBackward0>),
 tensor(1.0311, grad_fn=<NllLossBackward>))

In [21]:
model.adapt(adapt_loss[0])

In [29]:
F.softmax(logits, dim=1)

tensor([[0.0671, 0.1224, 0.1134, 0.1977, 0.4994],
        [0.0495, 0.0567, 0.0615, 0.0856, 0.7466],
        [0.0616, 0.1182, 0.0882, 0.0639, 0.6681],
        [0.1082, 0.1271, 0.1970, 0.1099, 0.4578],
        [0.1577, 0.2765, 0.2085, 0.1579, 0.1994],
        [0.2518, 0.1493, 0.2866, 0.1699, 0.1425],
        [0.1147, 0.1307, 0.3507, 0.2322, 0.1718],
        [0.1675, 0.1424, 0.3328, 0.1532, 0.2042],
        [0.1277, 0.1921, 0.2469, 0.1764, 0.2570],
        [0.1496, 0.2345, 0.2737, 0.1282, 0.2140],
        [0.2273, 0.2629, 0.1814, 0.1745, 0.1539],
        [0.2630, 0.2246, 0.0819, 0.1546, 0.2759],
        [0.3271, 0.1557, 0.1397, 0.2652, 0.1123],
        [0.3151, 0.2511, 0.1084, 0.1540, 0.1715],
        [0.2765, 0.1391, 0.1244, 0.3009, 0.1591],
        [0.2782, 0.2093, 0.2391, 0.1639, 0.1095],
        [0.2384, 0.2250, 0.1913, 0.1731, 0.1722],
        [0.1471, 0.3402, 0.1831, 0.1211, 0.2085],
        [0.2134, 0.2904, 0.2023, 0.1475, 0.1463],
        [0.1804, 0.2893, 0.1912, 0.1854, 0.1536],


In [22]:
task = train_tasks.sample()
model = learner.clone()
evaluation_loss, evaluation_accuracy = inner_adapt_delpo(
            task, reconst_loss, model, n_ways, k_shots, q_shots, inner_adapt_steps_train, device)

In [ ]:
evaluation_loss

Error: Kernel is dead

In [21]:
evaluation_loss[0].backward()
meta_train_loss.append(evaluation_loss[0])
meta_train_acc.append(evaluation_accuracy)


In [11]:
evaluation_loss[0]

tensor(16.9887, grad_fn=<AddBackward0>)

In [ ]:
for p in learner.parameters():
    p.grad.data.mul_(1.0 / 3)
opt.step()

In [7]:
for iter in range(10):
    opt.zero_grad()
    meta_train_losses = []
    meta_valid_losses = []
    meta_train_acc = []
    meta_valid_acc = []

    for batch in range(5):
        ttask = train_tasks.sample()
        model = learner.clone()
        evaluation_loss, evaluation_accuracy = inner_adapt_delpo(
            ttask, reconst_loss, model, n_ways, k_shots, q_shots, inner_adapt_steps_train, device)
        evaluation_loss[0].backward()
        meta_train_losses.append([l.item() for l in evaluation_loss])
        meta_train_acc.append(evaluation_accuracy.item())

    vtask = valid_tasks.sample()
    model = learner.clone()
    validation_loss, validation_accuracy = inner_adapt_delpo(
        vtask, reconst_loss, model, n_ways, k_shots, q_shots, inner_adapt_steps_train, device)
    meta_valid_losses.append([l.item() for l in validation_loss])
    meta_valid_acc.append(validation_accuracy.item())

    for p in learner.parameters():
        p.grad.data.mul_(1.0 / 5)
    opt.step()

/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448278899/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [10]:
meta_train_losses

[[207.71360778808594,
  21.771520614624023,
  6.7143144607543945,
  2425.524658203125,
  1.5497252941131592],
 [194.03419494628906,
  21.9337100982666,
  5.416471481323242,
  2459.828857421875,
  1.4208571910858154],
 [199.2206268310547,
  30.40707015991211,
  5.68020486831665,
  2262.280517578125,
  1.405105471611023],
 [203.253662109375,
  22.707983016967773,
  5.878161430358887,
  2504.766357421875,
  1.4961985349655151],
 [195.3379364013672,
  20.966901779174805,
  7.109440326690674,
  2023.20703125,
  1.4702953100204468]]

In [9]:
y = np.hstack(meta_train_losses)
y

array([2.07713608e+02, 2.17715206e+01, 6.71431446e+00, 2.42552466e+03,
       1.54972529e+00, 1.94034195e+02, 2.19337101e+01, 5.41647148e+00,
       2.45982886e+03, 1.42085719e+00, 1.99220627e+02, 3.04070702e+01,
       5.68020487e+00, 2.26228052e+03, 1.40510547e+00, 2.03253662e+02,
       2.27079830e+01, 5.87816143e+00, 2.50476636e+03, 1.49619853e+00,
       1.95337936e+02, 2.09669018e+01, 7.10944033e+00, 2.02320703e+03,
       1.47029531e+00])

In [12]:
y[::5].mean()

199.91200561523436

In [14]:
y1 = np.array(meta_train_acc)
y1

array([0.29333332, 0.37333333, 0.42666668, 0.40000001, 0.34666666])